# House Prices: Advanced Regression Techniques

> Date:2018-10-29,Author:Ho Loong

1. **Define the Problem:** If data science, big data, machine learning, predictive analytics, business intelligence, or any other buzzword is the solution, then what is the problem? As the saying goes, don't put the cart before the horse. Problems before requirements, requirements before solutions, solutions before design, and design before technology. Too often we are quick to jump on the new shiny technology, tool, or algorithm before determining the actual problem we are trying to solve.
2. **Gather the Data:** John Naisbitt wrote in his 1984 (yes, 1984) book Megatrends, we are “drowning in data, yet staving for knowledge." So, chances are, the dataset(s) already exist somewhere, in some format. It may be external or internal, structured or unstructured, static or streamed, objective or subjective, etc. As the saying goes, you don't have to reinvent the wheel, you just have to know where to find it. In the next step, we will worry about transforming "dirty data" to "clean data."
3. **Prepare Data for Consumption:** This step is often referred to as data wrangling, a required process to turn “wild” data into “manageable” data. Data wrangling includes implementing data architectures for storage and processing, developing data governance standards for quality and control, data extraction (i.e. ETL and web scraping), and data cleaning to identify aberrant, missing, or outlier data points.
4. **Perform Exploratory Analysis:** Anybody who has ever worked with data knows, garbage-in, garbage-out (GIGO). Therefore, it is important to deploy descriptive and graphical statistics to look for potential problems, patterns, classifications, correlations and comparisons in the dataset. In addition, data categorization (i.e. qualitative vs quantitative) is also important to understand and select the correct hypothesis test or data model.
5. **Model Data:** Like descriptive and inferential statistics, data modeling can either summarize the data or predict future outcomes. Your dataset and expected results, will determine the algorithms available for use. It's important to remember, algorithms are tools and not magical wands or silver bullets. You must still be the master craft (wo)man that knows how-to select the right tool for the job. An analogy would be asking someone to hand you a Philip screwdriver, and they hand you a flathead screwdriver or worst a hammer. At best, it shows a complete lack of understanding. At worst, it makes completing the project impossible. The same is true in data modelling. The wrong model can lead to poor performance at best and the wrong conclusion (that’s used as actionable intelligence) at worst.
6. **Validate and Implement Data Model:** After you've trained your model based on a subset of your data, it's time to test your model. This helps ensure you haven't overfit your model or made it so specific to the selected subset, that it does not accurately fit another subset from the same dataset. In this step we determine if our [model overfit, generalize, or underfit our dataset](http://docs.aws.amazon.com/machine-learning/latest/dg/model-fit-underfitting-vs-overfitting.html).
7. **Optimize and Strategize:** This is the "bionic man" step, where you iterate back through the process to make it better...stronger...faster than it was before. As a data scientist, your strategy should be to outsource developer operations and application plumbing, so you have more time to focus on recommendations and design. Once you're able to package your ideas, this becomes your “currency exchange" rate.

# Step 1:Define Problem

[Kaggle:House Prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

* Supervise/Unsupervise:Supervise
* Classification/Regression:Regression
* Eval:RMSE
* Train Data:'./input/train.csv'
* Test Data:'./input/test.csv'

# Step 2:Gather the Data

# Step 3:Prepare Data for Consumption

## 3.1 Import Common Libs

### 3.11 Import Model Libs

## 3.2 Meet and Greet Data

### 3.21 The 4 C`s of Data Cleaning:Correcting, Completing, Creating and Conveting

### 3.22 Clean Data

### 3.23 Convert Format

### 3.24 Double Check Cleaned Data

### 3.25 Split Data to Train and Test

# Step 4:Perform Exloratory Analysis with Statistics

# Step 5:Model Data

## 5.1 Evaluate Model Performance

### 5.11 Model Performance with Cross-Validation (CV)

### 5.12 Tune Model with Hyper-Parameters

### 5.13 Tune Model with Feature Selection

# Step 6:Validate and Implement

# Step 7: Optimize and Strategize